In [18]:
from typing import List
import copy
import pennylane as qml
import pennylane.numpy as np
from mitiq.zne.inference import RichardsonFactory
from mitiq.zne.scaling import fold_global
import logging
import qiskit

import util


SIMULATION_MODE = True    # running from local or real hardware
noise_strengths = np.linspace(0.1,1.0,20)
scale_factors = range(1,5)
ibmqx_token="xxx"

Zero Noise Extrapolation (ZNE) is a powerful technique in quantum computing aimed at mitigating errors arising from noise and imperfections in quantum hardware. By leveraging measurements from noisy quantum circuits, ZNE algorithms can estimate and correct errors, ultimately enhancing the reliability and accuracy of quantum computations.

In this notebook, I 
- Implement ZNE from scratch
- Compare with the result of Mitiq implementation 
- Experiment with ZNE algorithms to optimize its hyperparameters

##  Build a simple noise model with depolarizing noise

In [8]:
if SIMULATION_MODE:
    dev_noise_free = qml.device("default.mixed", wires=2)
    devs = [qml.transforms.insert(dev_noise_free,
                                  qml.DepolarizingChannel,
                                  noise_strength) 
            for noise_strength in noise_strengths]
else:
    dev = qml.device('qiskit.ibmq', wires=2, 
                     backend='ibm_kyoto', ibmqx_token=ibmqx_token)
noise_less_dev = qml.device("default.qubit", wires=2)

In [9]:
def circuit():
    """
    @A circuit preparing a Bell state
    """
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))

## Applying unitary folding method

In [10]:
def circuit_from_ops(operations: List, measurements: List):
    with qml.Tracker(dev) as tracker:
        tracker.active = True
        for op in operations:
            qml.apply(op)
            logging.info(tracker.latest)
    return qml.apply(measurements[0])

In [ ]:
results = {}

`results` will have the following structure
```
{
  'noise_level_1': {
    'linear': [list of results by folding
    'polinomial'
    'exponential'
    'reference_polinomial_
   }
}
```

In [11]:
results = []
for dev in devs:
    device_circuit = qml.QNode(circuit, dev)    
    for scale_factor in scale_factors:
        ops, measurements = util.unitary_fold(device_circuit, scale_factor)
        folded_circuit = qml.QNode(circuit_from_ops, dev)
        results.append(folded_circuit())

INFO - Pass: ContainsInstruction - 0.02718 (ms)
INFO - Pass: ConvertConditionsToIfOps - 0.04578 (ms)
INFO - Pass: UnitarySynthesis - 0.01311 (ms)
INFO - Pass: HighLevelSynthesis - 0.15092 (ms)
INFO - Begin BasisTranslator from source basis {('h', 1), ('cx', 2), ('measure', 1)} to target basis {'multiplexer', 'mcswap', 'rz', 'swap', 'u', 'mcrz', 'initialize', 'u1', 'ry', 'mcu1', 'mcz', 'id', 'x', 'ccx', 'u2', 'cp', 'mcu3', 'measure', 'rx', 'mcsx', 'h', 'mcp', 'p', 'diagonal', 'mcrx', 'ryy', 'mcr', 'cy', 'mcy', 'roerror', 'u3', 'mcry', 'snapshot', 'y', 'cu1', 'cz', 'rzx', 'barrier', 's', 'tdg', 'csx', 't', 'cx', 'cu2', 'r', 'z', 'cu3', 'rxx', 'unitary', 'delay', 'rzz', 'sdg', 'cswap', 'mcu2', 'sx', 'mcx', 'kraus'}.
INFO - Basis translation path search completed in 0.000s.
INFO - Basis translation paths composed in 0.001s.
INFO - Basis translation instructions replaced in 0.000s.
INFO - Pass: BasisTranslator - 3.60894 (ms)
INFO - Pass: Depth - 0.06080 (ms)
INFO - Pass: FixedPoint - 0.0188

## Extrapolation

In [12]:
extrapolate = {}
extrapolate['linear'] = util.linear_extrapolation(scale_factors, results)
extrapolate['polinomial'] = util.polinomial_extraplation(scale_factors, results, len(scale_factors)-1)
extrapolate['exp'] = util.exponential_extraplation(scale_factors, results)

## Create a reference ZNE mitigated circuit

In [13]:
# reference implementation
error_mitigated_device_circuit = qml.transforms.mitigate_with_zne(
    device_circuit,
    scale_factors,
    folding = fold_global,    
    extrapolate = RichardsonFactory.extrapolate,
)

## Compare the results between noisy circuit, reference method, and implemented method

In [14]:
unmitigated = device_circuit()
mitigated = error_mitigated_device_circuit()
noise_free = qml.QNode(circuit, noise_less_dev)()
print(f"Unmitigated result {unmitigated}")
print(f"Noise free (ideal) result  {noise_free}")
print()
print(f"Linear extrapolation: {extrapolate['linear']}")
print()
print("Polynomial extrapolation:")
print(f"\tReference mitigated result   {mitigated}")
print(f"\tImplemented mitigated result   {extrapolate['polinomial']}")
print()
print(f"Exp extrapolation: {extrapolate['exp']}")

INFO - Pass: ContainsInstruction - 0.01812 (ms)
INFO - Pass: ConvertConditionsToIfOps - 0.03672 (ms)
INFO - Pass: UnitarySynthesis - 0.01407 (ms)
INFO - Pass: HighLevelSynthesis - 0.09894 (ms)
INFO - Begin BasisTranslator from source basis {('h', 1), ('cx', 2), ('measure', 1)} to target basis {'multiplexer', 'mcswap', 'rz', 'swap', 'u', 'mcrz', 'initialize', 'u1', 'ry', 'mcu1', 'mcz', 'id', 'x', 'ccx', 'u2', 'cp', 'mcu3', 'measure', 'rx', 'mcsx', 'h', 'mcp', 'p', 'diagonal', 'mcrx', 'ryy', 'mcr', 'cy', 'mcy', 'roerror', 'u3', 'mcry', 'snapshot', 'y', 'cu1', 'cz', 'rzx', 'barrier', 's', 'tdg', 'csx', 't', 'cx', 'cu2', 'r', 'z', 'cu3', 'rxx', 'unitary', 'delay', 'rzz', 'sdg', 'cswap', 'mcu2', 'sx', 'mcx', 'kraus'}.
INFO - Basis translation path search completed in 0.000s.
INFO - Basis translation paths composed in 0.001s.
INFO - Basis translation instructions replaced in 0.000s.
INFO - Pass: BasisTranslator - 2.60687 (ms)
INFO - Pass: Depth - 0.03719 (ms)
INFO - Pass: FixedPoint - 0.0121

Unmitigated result 1.0
Noise free (ideal) result  0.9999999999999998

Linear extrapolation: 0.9999999999999999

Polynomial extrapolation:
	Reference mitigated result   0.9999999999999991
	Implemented mitigated result   0.9999999999999991

Exp extrapolation: 1.0


## Reserach questions

It is natural to ask which method and which fold to use when. To answer that we 